## **Revisión de funciones de gradiente y hessiana para sistema agrandado**

**Responsable:** Yalidt Díaz Vázquez

**Infraestructura utilizada:** La implementación de la revisión se llevó a cabo en google colab.

**Objetivo** : Implementar el código ya desarollado en *numpy* a *cupy* para poder utilizarlo en paralelo y resolver el portafolio de **Marcowitz** con el método de Netwton.

In [1]:
!pip3 install --user cupy-cuda102

     |████████████████████████████████| 380.4MB 38kB/s 


In [2]:
#Librerías utilizadas
import cupy as cp
import numpy as np
import pandas as pd
import fix_yahoo_finance as yf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import time
import funciones_auxiliares

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Test 1 : matriz de maxímo 10**2

In [0]:
np.random.seed(141394) #Implementamos una semilla
n=10**2
#Generamos un vector con entradas aleatorias para simlular el precio de las acciones
#X_test=np.random.uniform(1,n,size=(55,55))
X_test=cp.random.rand(n, n)
#Calcularmos la varianza de los datos simulados
Sigma=funciones_auxiliares.calcular_varianza(X_test)
mu=funciones_auxiliares.calcular_rendimiento(X_test)

#Definimos el retorno máximo
r=max(mu)

In [77]:
Sigma.shape

(100, 100)

In [78]:
mu.shape

(100,)

Calculamos el gradiente simbólico

In [0]:
#Se define la función para el calculo del gradiente
def gfo_cp_mark(x):
    v = x[0:(x.size-2)]
    value1 = x[x.size-2]
    value2 = x[x.size-1]
    #m,n=Sigma.shape
    first_block = Sigma@v -1/2*value1*mu -1/2*value2*unos
    second_block = -1/2*(v.dot(mu)-r)
    third_block = -1/2*(v.dot(unos)-1)
    first_block_np = cp.asnumpy(first_block)
    second_block_np = cp.asnumpy(second_block)
    third_block_np = cp.asnumpy(third_block)
    #es necesario pasar los bloques a numpy para poder hacerles concatenate 
    return cp.asarray(np.concatenate((first_block_np,np.array([second_block_np]),np.array([third_block_np]))))

In [80]:
#Generamos los vectores auxiliares y verificamos tenga la dimensión correcta
unos = cp.ones(n)
unos.shape

(100,)

In [81]:
x = cp.ones(n+2)
x.shape

(102,)

In [82]:
gradiente=gfo_cp_mark(x)
gradiente

array([  5.59389083,   4.16298669,   1.16173511,  -0.80452193,
         8.06850161,   4.3992134 ,  -4.37988936,  -3.47449534,
         3.08104278,   0.27422126,   1.50603361,  -1.75408277,
         5.65452205,   0.10250282,   0.77642265,   0.63016518,
         0.17076949,   4.54840769,   2.90662153,   7.27628177,
        -1.06703195,  -0.27907012,   2.26378022,   3.10674916,
         4.49429706,   0.64247775,   5.64809942,  -1.01016255,
         2.32326918,  -5.3998149 ,   3.95431778,   2.72399494,
         0.4537366 ,  -0.63625346,  -0.06449992,  -0.43771631,
         2.26618511,   4.90680216,   5.12511457,  -1.218305  ,
        -3.67054273,   0.64530235,  -0.6721487 ,  -0.77244143,
        -4.30119408,   5.47597868,   1.97266713,   1.9916272 ,
         0.47649751,  -1.22048931,  -2.74664156,  -4.52301709,
         2.53909588,   8.68880158,   4.18834463,   4.94865022,
         1.12245783,  -3.51087556,  -3.17834954,  -0.69359037,
         3.85684406,   2.48662992,  -0.93311724,   1.85

Calculamos el hessiana simbólica

In [0]:
def Hfo_cp_mark(x):
    v = x[0:(x.size-2)]
    value1 = x[x.size-2]
    value2 = x[x.size-1]
    m,n=Sigma.shape
    first_block = Sigma 
    second_block = -1/2*mu
    third_block = -1/2*unos
    fs_block = cp.column_stack((first_block, second_block, third_block))
    fourth_block = -1/2*mu
    fifth_block = -1/2*unos
    sixth_block = cp.zeros((2,2))
    #es necesario pasar los bloques a numpy para poder hacerles concatenate y row_stack
    tf_block = np.column_stack(( np.row_stack((cp.asnumpy(fourth_block),cp.asnumpy(fifth_block))),cp.asnumpy(sixth_block)))
    return cp.asarray(np.row_stack((cp.asnumpy(fs_block), tf_block)))

In [84]:
hessiana=Hfo_cp_mark(x)
hessiana

array([[ 1.8450889 , -0.13067072,  0.45438903, ..., -0.10268936,
         0.28316019, -0.5       ],
       [-0.13067072,  1.95291512,  0.05686187, ..., -0.01609178,
         1.578899  , -0.5       ],
       [ 0.45438903,  0.05686187,  1.27961467, ..., -0.3432405 ,
        -0.00612039, -0.5       ],
       ...,
       [-0.10268936, -0.01609178, -0.3432405 , ...,  2.45811277,
         2.7469915 , -0.5       ],
       [ 0.28316019,  1.578899  , -0.00612039, ...,  2.7469915 ,
         0.        ,  0.        ],
       [-0.5       , -0.5       , -0.5       , ..., -0.5       ,
         0.        ,  0.        ]])

Test 2 : matriz de maxímo 10**3

In [0]:
np.random.seed(141394) #Implementamos una semilla
n=10**3
#Generamos un vector con entradas aleatorias para simlular el precio de las acciones
#X_test=np.random.uniform(1,n,size=(55,55))
X_test=cp.random.rand(n, n)
#Calcularmos la varianza de los datos simulados
Sigma=funciones_auxiliares.calcular_varianza(X_test)
mu=funciones_auxiliares.calcular_rendimiento(X_test)

#Definimos el retorno máximo
r=max(mu)

#Vectores auxiliares
unos = cp.ones(n)
x = cp.ones(n+2)

In [88]:
gradiente=gfo_cp_mark(x)
gradiente

array([  -1.23443998,    1.35120825,    1.08749611, ...,    2.09910078,
         10.3089411 , -499.5       ])

In [89]:
hessiana=Hfo_cp_mark(x)
hessiana

array([[ 1.7765566 , -0.00853804, -0.11948184, ...,  0.05922019,
        -0.01188587, -0.5       ],
       [-0.00853804,  2.04182097, -0.05446197, ...,  0.02523144,
         0.02635109, -0.5       ],
       [-0.11948184, -0.05446197,  2.33995493, ...,  0.03759729,
        -0.04627767, -0.5       ],
       ...,
       [ 0.05922019,  0.02523144,  0.03759729, ...,  2.06444492,
        -0.05266467, -0.5       ],
       [-0.01188587,  0.02635109, -0.04627767, ..., -0.05266467,
         0.        ,  0.        ],
       [-0.5       , -0.5       , -0.5       , ..., -0.5       ,
         0.        ,  0.        ]])

Test 3 : matriz de maxímo 10**4

In [0]:
np.random.seed(141394) #Implementamos una semilla
n=10**4
#Generamos un vector con entradas aleatorias para simlular el precio de las acciones
#X_test=np.random.uniform(1,n,size=(55,55))
X_test=cp.random.rand(n, n)
#Calcularmos la varianza de los datos simulados
Sigma=funciones_auxiliares.calcular_varianza(X_test)
mu=funciones_auxiliares.calcular_rendimiento(X_test)

#Definimos el retorno máximo
r=max(mu)

#Vectores auxiliares
unos = cp.ones(n)
x = cp.ones(n+2)

In [91]:
gradiente=gfo_cp_mark(x)
gradiente

array([ 8.72898925e-01,  1.22191553e+00,  5.50440683e-01, ...,
        5.70651989e+00,  8.13396989e-01, -4.99950000e+03])

In [92]:
hessiana=Hfo_cp_mark(x)
hessiana

array([[ 1.98746995e+00, -1.69016294e-02, -1.57471409e-02, ...,
        -3.73195157e-03,  4.86941944e-03, -5.00000000e-01],
       [-1.69016294e-02,  2.09082863e+00,  4.43902497e-03, ...,
         2.08907771e-02, -3.54055846e-04, -5.00000000e-01],
       [-1.57471409e-02,  4.43902497e-03,  2.06073635e+00, ...,
         1.34622766e-02,  2.94406209e-02, -5.00000000e-01],
       ...,
       [-3.73195157e-03,  2.08907771e-02,  1.34622766e-02, ...,
         2.08580717e+00,  3.14277675e-03, -5.00000000e-01],
       [ 4.86941944e-03, -3.54055846e-04,  2.94406209e-02, ...,
         3.14277675e-03,  0.00000000e+00,  0.00000000e+00],
       [-5.00000000e-01, -5.00000000e-01, -5.00000000e-01, ...,
        -5.00000000e-01,  0.00000000e+00,  0.00000000e+00]])

**Hallazgos**

1)Funciona bien para dimensiones grandes de matrices, incluso con 10**4

**Mejoras**

1) Falta documentación de ambas funciones <br>
2) Meter como parámetros a ambas funcioones : Sigma, r, mu e incluso la dimension de los vectores de *unos* para que estos esten definidos dentro de la función y solo reciban como parámetos la dimensión.